# 🦸🏻 Comprendiendo el Semantic Kernel con Héroes 🦸🏼‍♀️

Semantic Kernel ha sido creado para permitir a los desarrolladores integrar Inteligencia Artificial en sus aplicaciones de manera fluida. Para lograr esto, proporciona un conjunto de características que te permitirán añadir modelos, prompts, funciones nativas y memorias sin requerir un conocimiento profundo de IA 🥲. Por eso se dice que Semantic Kernel simula el cerebro 🧠 de tu aplicación.

<div style="text-align:center">
    <img src="images/semantic-kernel.png" width="15%">
</div>

## (Opcional) - Creando un Servicio Azure Open AI 🤖 y Despliegues

Antes de sumergirnos en Semantic Kernel 🛝, necesitarás tener configurado uno de los servicios soportados. Actualmente, puedes elegir entre las siguientes opciones: [Azure Open AI](https://azure.microsoft.com/en-us/products/ai-services/openai-service), [Open AI](https://openai.com/), o [Hugging Face](https://huggingface.co/).

En este ejemplo, voy a usar Azure Open AI.

Por lo tanto, a través de Azure CLI, necesito iniciar sesión:


In [ ]:
az login

Si lo necesitas, porque tienes muchas suscripciones en tu cuenta, puedes seleccionar la que te interesa (o la que tiene acceso a Azure Open AI 😊) a través del siguiente comando:


In [ ]:
az account set -n "Visual Studio Enterprise Subscription"

Ahora, para poder crear lo que necesitas para este cuaderno, establece las siguientes variables con tus valores preferidos:


In [ ]:
$RESOURCE_GROUP="understanding-semantic-kernel"
$LOCATION="canadaeast"
$AZURE_OPEN_AI="ai-for-heroes"

Con ellos, ahora puedes crear el grupo de recursos:


In [ ]:
az group create `
--name $RESOURCE_GROUP `
--location $LOCATION

Un recurso de Azure Open AI:


In [ ]:
az cognitiveservices account create `
--name $AZURE_OPEN_AI `
--custom-domain $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--kind OpenAI `
--sku S0 `
--location $LOCATION

Y finalmente, necesitas un despliegue de cualquiera de los modelos que tienes disponibles. En este ejemplo, voy a usar gpt-4:

In [ ]:
az cognitiveservices account deployment create `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--deployment-name "gpt-4" `
--model-name "gpt-4" `
--model-version "0613"  `
--model-format OpenAI `
--sku-capacity "10" `
--sku-name "Standard"

El parámetro **sku-capacity** nos permite especificar cuántos tokens por minuto podemos enviar a este modelo. Para ver cómo se está utilizando tu cuota, puedes usar este otro comando:


In [ ]:
az cognitiveservices usage list `
-l $LOCATION

Carga las variables de entorno con tu endpoint y clave de Azure Open AI:

In [ ]:
$env:AZURE_OPEN_AI_KEY =$(az cognitiveservices account keys list `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--query "key1" `
--output tsv)

$env:AZURE_OPEN_AI_ENDPOINT =$(az cognitiveservices account show `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--query "properties.endpoint" `
--output tsv)

#dir env:AZURE_OPEN_AI_KEY
#dir env:AZURE_OPEN_AI_ENDPOINT

## Cómo empezar con Semantic Kernel


Lo primero que necesitas para poder ejecutar Semantic Kernel en este cuaderno es instalar la biblioteca **Microsoft.SemanticKernel**, que actualmente está en la versión **1.5.0** (prometo seguir actualizando 🤓).

Puedes encontrar más información sobre esta biblioteca y sus actualizaciones en la página oficial de NuGet [aquí](https://www.nuget.org/packages/Microsoft.SemanticKernel).


In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.5.0"

Con ello, ahora puedes instanciar el cerebro de tu aplicación a través de **KernelBuilder**. Tiene muchos conectores a los modelos y otras cosas, pero por ahora, comencemos con lo básico:


In [ ]:
using Microsoft.SemanticKernel;

// Create a Builder
var builder = Kernel.CreateBuilder();

Dependiendo del tipo de tarea que quieras realizar, puedes usar diferentes métodos con el prefijo **With**. Por ahora, vamos a usar algunos de los modelos de tipo **completion** o completado.


In [ ]:
// OpenAI keys
var modelId = "gpt-4";
var apiKey = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Open AI key")).GetClearTextPassword();

// Create a chat completion service
builder.AddOpenAIChatCompletion(modelId, apiKey);

In [ ]:
// Azure OpenAI keys
var deploymentName = "gpt-4";
var endpoint = Environment.GetEnvironmentVariable("AZURE_OPEN_AI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("AZURE_OPEN_AI_KEY");

// Create a chat completion service
builder.AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey);

Con la configuración realizada, lo único que queda es generar el kernel con todo configurado:


In [ ]:
var kernel = builder.Build();

## Uso de plugins de Semantic Kernel

Los plugins son el núcleo de Semantic Kernel. Con ellos, encapsulas capacidades para que sean reutilizables, mantenibles y planificables (lo entenderás más tarde 🙃). Hay dos tipos: aquellos que consisten en plantillas de indicaciones llamadas **Semantic Functions** y funciones nativas del lenguaje de programación elegido llamadas **Native Functions**.

<div style="text-align:center">
    <img src="images/writer-plugin-example.png" width="40%" />
</div>

## Funciones Semánticas

Cuando hablas con modelos de inteligencia artificial, debes hacerlo con lo que se conoce como un *prompt*. Esto puede variar desde una simple frase hasta algo más elaborado, permitiendo que el modelo comprenda no solo lo que queremos sino también cómo lo queremos. Si echas un vistazo a la documentación oficial, define este tipo de función como la boca 👄 y los oídos 👂🏻 de tu cerebro 🧠. 🤖🌐

<div style="text-align:center">
    <img src="images/semantic-function-explainer.png" width="20%" />
</div>

Como parte de este repositorio, tienes una carpeta llamada **SemanticFunctions** que tiene diferentes funciones de este tipo:

- **FunPlugin**: Esto nos permite pedirle al modelo que haga bromas sobre héroes bajo ciertas condiciones, a través de la función **Joke**.
- **WritePlugin**: Para mostrarte que dentro de un plugin puedes tener diferentes funciones, en este directorio tenemos dos relacionadas con el arte de escribir: la primera, **OOF**, nos permite generar el mensaje "Fuera de la oficina" 🏢📧 para superhéroes y la segunda **StoryGen** nos ayudará a crear historias, también sobre superhéroes 🦸🏻‍♂️🦸🏻‍♀️.

Para que nuestro kernel sepa que estos plugins están disponibles, primero necesitas obtener la ruta del directorio:


In [ ]:
using System.IO;

var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SemanticFunctions", "FunPlugIn");

Y luego comienza a cargar los plugins que deseas:


In [ ]:
// Load the FunPlugin from the Plugins Directory
var funPluginFunctions = kernel.ImportPluginFromPromptDirectory(pluginsDirectory);

Para poder invocar una función de este tipo, puedes hacerlo de la siguiente manera:


In [ ]:
var joke = new KernelArguments() { ["input"] = "tell me a joke" };
var result = await kernel.InvokeAsync(funPluginFunctions["Joke"], joke);

Como puedes ver, solo usando **kernel.RunAsync** y pasando mi solicitud como parámetros y qué función, en este caso **Joke**, dentro del plugin, en este caso **FunPlugin**, quiero usar.

Cada una de estas funciones consta de dos archivos:

- **skprompt.txt** es el archivo donde se define el prompt para enviar al modelo en un formato de plantilla, de modo que pueda recibir parámetros y hacerlos más reutilizables.


    ```
    WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE SUBJECT BELOW

    JOKE MUST BE:
    - IN ENGLISH

    BE CREATIVE AND FUNNY. I WANT TO LAUGH.

    Incorporate the hero if provided: {{$hero}}
    +++++

    {{$input}}
    +++++
    ```

- **config.json**, which allows us to indicate the maximum number of tokens allowed for this call (**max_tokens**), the temperature to control the randomness of the responses (**temperature**), which means that closer to 1 they will be more random and closer to zero more determined and focused on the most likely response, **top_p** is used to control the diversity of the responses, where a value of 0.0 means that only the most likely responses will be considered and 1 where all possible responses will be considered and **presence_penalty** and **frequency_penalty** to adjust the penalty for the presence and frequency of the tokens in the generated responses. On the other hand, if the skprompt.txt file receives parameters, they must also be defined in this file in the **parameters** array of the **input** object.

  ```javascript
  {
    "schema": 1,
    "description": "Generate a funny joke about heroes",
    "models": [
      {
        "max_tokens": 150,
        "temperature": 0.9,
        "top_p": 0.5,
        "presence_penalty": 0.2,
        "frequency_penalty": 0.3
      }
    ],
    "input": {
      "parameters": [
        {
          "name": "input",
          "description": "Joke subject",
          "defaultValue": ""
        },
        {
          "name": "hero",
          "description": "Give a hint about the hero you want to joke about",
          "defaultValue": ""
        }
      ]
    }
  }
  ```


Para ver el resultado generado al llamar a esta función Joke, puedes obtenerlo de la siguiente manera:


In [ ]:
Console.WriteLine(result.GetValue<string>());

As you can see, in this execution we have only passed the input parameter, but not the hero about whom we wanted to generate this joke. To be able to send more than one parameter you need to create an object of the **ContextVariables** type:


In [ ]:
var variables = new KernelArguments() {
    ["input"] = "Tell me a Christmas joke",
    ["hero"] = "Ironman"
};

To be able to use these two values as part of the call, simply put it as the first argument.


In [ ]:
var result = await kernel.InvokeAsync(funPluginFunctions["Joke"], variables);

Now check if the joke is about the hero specified as a parameter.


In [ ]:
Console.WriteLine(result.GetValue<string>());

In the same way, we can use the functions included in **WriterPlugin**:


In [ ]:
// Load the WriterPlugin from the Plugins Directory
var pluginsDirectoryWriter = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SemanticFunctions", "WriterPlugin");

var writerPluginFunctions = kernel.ImportPluginFromPromptDirectory(pluginsDirectoryWriter);


One that generates stories:


In [ ]:
var variables = new KernelArguments() {
    ["input"] = "Tell me a Christmas Story",
    ["hero"] = "Ironman"
};

var result = await kernel.InvokeAsync(writerPluginFunctions["StoryGen"], variables);
Console.WriteLine(result);

Or even to create messages for when the heroes 🦸🏻‍♀️ are on vacation ✈️🚢🌴:


In [ ]:
var variables = new KernelArguments() {
    ["input"] = "Create an OOF for Christmas",
    ["hero"] = "Hulk"
};

var result = await kernel.InvokeAsync(writerPluginFunctions["OOF"], variables);
Console.WriteLine(result.GetValue<string>());

## Native Functions

While semantic functions allow us to define and reuse prompts, **with native functions you can make the semantic kernel call functions written in C# or Python**, for tasks that go beyond a call through a prompt.

<div style="text-align:center">
    <img src="images/native-function-explainer.png" width="25%">
</div>

### Why do I need native functions in these types of applications?

Large language models (LLMs) are excellent for generating text, but there are several tasks they cannot perform on their own. These include, among others:

- Retrieving data from external data sources
- Knowing what time it is
- Performing complex mathematical operations
- Completing tasks in the real world
- Memorizing and remembering information

For these scenarios, and many others, native functions are very useful 👍🏻


For this example, I'm going to use an API called **SuperHero API**, which requires an API key. You can get it from their website: [https://superheroapi.com/](https://superheroapi.com/)

Once you have it, pass it to the prompt that appears with the following line:


In [ ]:
var superHeroApiKey = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Super Hero Api key")).GetClearTextPassword();

Now, to load a native function, we must add this before we build the Kernel.

Let's create a new Super Hero Info class and add it as a Plugin.

In [ ]:
#load "NativeFunctions/HeroInfo.cs"

var heroInfo = new HeroInfo(superHeroApiKey);
builder.Plugins.AddFromObject(heroInfo, "HeroInfo");
Kernel kernel = builder.Build();

The builder now can interact with the local function. Let's make a call to get the alter ego of a super hero.

In [ ]:
var kernelArgs = new KernelArguments()
{
    ["input"] = "Ironman"
};

var result = await kernel.InvokeAsync<string>("HeroInfo", "GetAlterEgo", kernelArgs);
Console.WriteLine(result);

In this repo, there is another directory called **NativeFunctions** where you can find a class called **HeroInfo.cs**. In it, there is a function decorated with the **KernelFunction** attribute, which allows us to indicate to the Semantic Kernel that it is a native function and, through the **Description** property, also give it information about what the purpose of this function is. In this case, what this method allows us to do is retrieve the alter ego of the superhero that we pass as a parameter, in this example that of Ironman. 


## Planner

So far, all the plugins you have seen have been executed intentionally. That is, no one has chosen them for you and you can run them based on your needs. However, this is the most *static* way to interact with Semantic Kernel. There is another option called **Planner** that will leave you astounded 😮

Planner is a function that takes a user's request and returns a plan on how to carry out the request. To do this, it uses AI to combine the plugins registered in the core and recombine them into a series of steps that complete a goal.

<div style="text-align:center;">
    <img src="images/the-planner.png" width=35% />
</div>

To see it in action, we are going to use the plugins that you already know.

The first thing you need is to add the nuget reference to the Planner Handlebars.

To learn more about this NuGet package, you can visit [here](https://www.nuget.org/packages/Microsoft.SemanticKernel.Planners.Handlebars/1.5.0-preview#show-readme-container).



In [ ]:
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.5.0-preview"

Now we are ready to create a planner.

In [ ]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner();


There are different types as you can see [here](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/planners/?tabs=Csharp). In this example, we are going to use the one that executes tasks sequentially.

Since you already have all the plugins loaded in your kernel instance, we can ask it something like this:


In [ ]:
using System.Text.Json;

var ask = "I would like you to tell me a joke about Batman, and with that joke, create an out-of-office message using the joke.";
#pragma warning disable SKEXP0060
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

As you can see, the planner knows about the  plugins thanks to the description that I included as part of their implementation.

Time to run the plan and get an OOF for **Batman** that includes a **bad joke**.


In [ ]:
// executing the plan
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(originalPlanResult.ToString());

### Advanced Planner Labs

The kernel already has the plugins loaded, and the function, let's see if it can solve a more complex example:


In [ ]:
var ask_complex = "I would like you to find out IronMan's alter ego, and create an out of office message for IronMan, signing it with his alter ego.";

#pragma warning disable SKEXP0060
var plan_complex = await planner.CreatePlanAsync(kernel, ask_complex);

Console.WriteLine("Plan:\n");
Console.WriteLine(plan_complex);

Let's execute the plan and see its output:


In [ ]:
#pragma warning disable SKEXP0060
var complexPlanResult = await plan_complex.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Complex  Plan results:\n");
Console.WriteLine(complexPlanResult.ToString());

## Kernel Memory

<img src="images/How kernel memory works.png" width="80%" />

To be able to use Kernel Memory, you need to add its nuget library, in addition to importing the class that I have generated in the **KernelMemory** directory.

To learn more about the Kernel Memory library, you can visit the [NuGet package page](https://www.nuget.org/packages/Microsoft.SemanticKernel.Plugins.Memory/1.5.0-alpha#show-readme-container).


In [ ]:
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.5.0-alpha"
#r "nuget: System.Linq.Async, 6.0.1"


In order to use memory, we need to instantiate the Memory Plugin with a Memory Storage and an Embedding backend. In this example, we make use of the VolatileMemoryStore which can be thought of as a temporary in-memory storage (not to be confused with Semantic Memory).

This memory is not written to disk and is only available during the app session.

When developing your app you will have the option to plug in persistent storage like Azure Cosmos Db, PostgreSQL, SQLite, etc. Semantic Memory allows also to index external data sources, without duplicating all the information, more on that later.

In [ ]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// create the memory builder
#pragma warning disable SKEXP0003, SKEXP0011, SKEXP0052
var memoryBuilder = new MemoryBuilder();

// Using OpenAI
memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey);

// Using Azure OpenAI 
// memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
//         AzureOpenAI.EmbeddingsModel,
//          AzureOpenAI.Endpoint,
//         AzureOpenAI.ApiKey,
//         "text-embedding-ada-002");

memoryBuilder.WithMemoryStore(new VolatileMemoryStore());
var memory = memoryBuilder.Build();

Let's create some initial memories "Fan Facts". We can add memories to our VolatileMemoryStore by using SaveInformationAsync:



In [ ]:
// add fan facts to the collection
const string MemoryCollectionName = "fanFacts";

await memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "Gisela's favourite super hero is Batman");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "The last super hero movie watched by Gisela was Guardians of the Galaxy Vol 3");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "Bruno's favourite super hero is Invincible");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "The last super hero movie watched by Bruno was Aquaman II");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "Bruno don't like the super hero movie: Eternals");

And now we can test and search in the memory:

In [ ]:
var questions = new[]
{
    "what is Bruno's favourite super hero?",
    "what was the last movie watched by Gisela?",
    "Which is the prefered super hero for Gisela?",
    "Did Bruno watched a super hero movie in the past, which was the last one?"
};

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}
